# Lab | Data Structuring and Combining Data

## Challenge 1: Combining & Cleaning Data

In this challenge, we will be working with the customer data from an insurance company, as we did in the two previous labs. The data can be found here:
- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file1.csv

But this time, we got new data, which can be found in the following 2 CSV files located at the links below.

- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file2.csv
- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file3.csv

Note that you'll need to clean and format the new data.

Observation:
- One option is to first combine the three datasets and then apply the cleaning function to the new combined dataset
- Another option would be to read the clean file you saved in the previous lab, and just clean the two new files and concatenate the three clean datasets

In [6]:
# Your code goes here

import pandas as pd

#datasets
url1 = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file1.csv"
url2 = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file2.csv"
url3 = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file3.csv"

df1 = pd.read_csv(url1)
df2 = pd.read_csv(url2)
df3 = pd.read_csv(url3)

#combine datasets
df = pd.concat([df1, df2, df3], ignore_index=True)

#cleaning functions

def clean_gender(df):
    if "gender" in df.columns:
        df["gender"] = df["gender"].astype(str).str.upper()
        df["gender"] = df["gender"].replace({
            "M": "M", "MALE": "M",
            "F": "F", "FEMALE": "F", "FEMAL": "F"
        })
    return df

def clean_state(df):
    if "state" in df.columns:
        df["state"] = df["state"].replace({
            "AZ": "Arizona",
            "Cali": "California",
            "WA": "Washington"
        })
    return df

def clean_education(df):
    if "education" in df.columns:
        df["education"] = df["education"].replace({
            "Bachelors": "Bachelor"
        })
    return df

def clean_customer_lifetime_value(df):
    if "customer_lifetime_value" in df.columns:
        df["customer_lifetime_value"] = (
            df["customer_lifetime_value"]
            .astype(str)
            .str.replace("%", "", regex=False)
        )
        df["customer_lifetime_value"] = pd.to_numeric(df["customer_lifetime_value"], errors="coerce")
    return df

def clean_vehicle_class(df):
    if "vehicle_class" in df.columns:
        df["vehicle_class"] = df["vehicle_class"].replace({
            "Sports Car": "Luxury",
            "Luxury SUV": "Luxury",
            "Luxury Car": "Luxury"
        })
    return df

def clean_open_complaints(df):
    if "number_of_open_complaints" in df.columns:
        df["number_of_open_complaints"] = (
            df["number_of_open_complaints"]
            .astype(str)
            .str.split("/")
            .str[1]   #middle number from format like 1/5/00
        )
        df["number_of_open_complaints"] = pd.to_numeric(df["number_of_open_complaints"], errors="coerce")
    return df

def convert_numeric_to_int(df):
    for col in df.select_dtypes(include=["float", "int"]).columns:
        df[col] = df[col].fillna(0).astype(int)
    return df

#apply cleaning
df = clean_gender(df)
df = clean_state(df)
df = clean_education(df)
df = clean_customer_lifetime_value(df)
df = clean_vehicle_class(df)
df = clean_open_complaints(df)
df = convert_numeric_to_int(df)

#duplicates
df = df.drop_duplicates().reset_index(drop=True)

#save cleaned dataset
df.to_csv("cleaned_customer_data.csv", index=False)

#final checks
print("Cleaning complete!")
print("\nInfo")
print(df.info())
print("\nMissing values")
print(df.isnull().sum())
print("\nSample Data")
print(df.head())


Cleaning complete!

Info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9135 entries, 0 to 9134
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Customer                   9134 non-null   object
 1   ST                         2064 non-null   object
 2   GENDER                     1942 non-null   object
 3   Education                  9134 non-null   object
 4   Customer Lifetime Value    9127 non-null   object
 5   Income                     9135 non-null   int64 
 6   Monthly Premium Auto       9135 non-null   int64 
 7   Number of Open Complaints  9134 non-null   object
 8   Policy Type                9134 non-null   object
 9   Vehicle Class              9134 non-null   object
 10  Total Claim Amount         9135 non-null   int64 
 11  State                      7070 non-null   object
 12  Gender                     7070 non-null   object
dtypes: int64(3), object(10)
memory usage: 

# Challenge 2: Structuring Data

In this challenge, we will continue to work with customer data from an insurance company, but we will use a dataset with more columns, called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis_clean.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by performing data cleaning, formatting, and structuring.

In [7]:
# Your code goes here
import pandas as pd

#dataset
url = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis_clean.csv"
df = pd.read_csv(url)

print("Dataset loaded successfully!")
print("Shape:", df.shape)
print(df.head())

#total revenue by sales channel
pivot_sales_channel = (
    df.pivot_table(
        values="total_claim_amount",
        index="sales_channel",
        aggfunc="sum"
    )
    .round(2)
    .sort_values(by="total_claim_amount", ascending=False)
)

print("\n--- Total Revenue by Sales Channel ---")
print(pivot_sales_channel)

#average customer lifetime value by gender and education
pivot_clv = (
    df.pivot_table(
        values="customer_lifetime_value",
        index="gender",
        columns="education",
        aggfunc="mean"
    )
    .round(2)
)

print("\nAverage Customer Lifetime Value by Gender and Education")
print(pivot_clv)


Dataset loaded successfully!
Shape: (10910, 27)
   unnamed:_0 customer       state  customer_lifetime_value response  \
0           0  DK49336     Arizona              4809.216960       No   
1           1  KX64629  California              2228.525238       No   
2           2  LZ68649  Washington             14947.917300       No   
3           3  XL78013      Oregon             22332.439460      Yes   
4           4  QA50777      Oregon              9025.067525       No   

   coverage education effective_to_date employmentstatus gender  ...  \
0     Basic   College        2011-02-18         Employed      M  ...   
1     Basic   College        2011-01-18       Unemployed      F  ...   
2     Basic  Bachelor        2011-02-10         Employed      M  ...   
3  Extended   College        2011-01-11         Employed      M  ...   
4   Premium  Bachelor        2011-01-17    Medical Leave      F  ...   

   number_of_policies     policy_type        policy  renew_offer_type  \
0            

1. You work at the marketing department and you want to know which sales channel brought the most sales in terms of total revenue. Using pivot, create a summary table showing the total revenue for each sales channel (branch, call center, web, and mail).
Round the total revenue to 2 decimal points.  Analyze the resulting table to draw insights.

2. Create a pivot table that shows the average customer lifetime value per gender and education level. Analyze the resulting table to draw insights.

## Bonus

You work at the customer service department and you want to know which months had the highest number of complaints by policy type category. Create a summary table showing the number of complaints by policy type and month.
Show it in a long format table.

*In data analysis, a long format table is a way of structuring data in which each observation or measurement is stored in a separate row of the table. The key characteristic of a long format table is that each column represents a single variable, and each row represents a single observation of that variable.*

*More information about long and wide format tables here: https://www.statology.org/long-vs-wide-data/*

In [9]:
# Your code goes here
import pandas as pd

#dataset
url = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis_clean.csv"
df = pd.read_csv(url)

#make sure that the date column is datetime
df["effective_to_date"] = pd.to_datetime(df["effective_to_date"], errors="coerce")

#extract month name from the date
df["month"] = df["effective_to_date"].dt.month_name()

#summary table number of complaints by policy type and month
complaints_summary = (
    df.groupby(["policy_type", "month"])["number_of_open_complaints"]
    .count()
    .reset_index(name="complaints_count")
    .sort_values(["policy_type", "complaints_count"], ascending=[True, False])
)

print("\nComplaints by Policy Type and Month (Long Format)")
print(complaints_summary.head(20))



Complaints by Policy Type and Month (Long Format)
      policy_type     month  complaints_count
1  Corporate Auto   January              1252
0  Corporate Auto  February              1089
3   Personal Auto   January              4329
2   Personal Auto  February              3799
5    Special Auto   January               237
4    Special Auto  February               204
